In [4]:
#############
## imports ##
#############

# libraries 
import geopandas as gpd
import numpy as np 
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.mask import mask
import os
import json
import logging

# functions 
from functions import filter_imgs, Crop_tif

##################
## setup logger ##
##################

logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)

# Setup logger
ch = logging.StreamHandler() # create console handler
ch.setLevel(logging.DEBUG) # set level to debug
formatter = logging.Formatter("%(asctime)s - %(message)s \n", "%Y-%m-%d %H:%M:%S") # create formatter
ch.setFormatter(formatter) # add formatter to ch
logger.addHandler(ch) # add ch to logger

# logger.info('Imports successful')

##################
## load configs ##
##################

config_path = os.path.join(os.getcwd(), 'configs.json')
with open(config_path, 'r') as config_file:
    configs = json.load(config_file)

# load paths from configs 
config_paths = configs.get('paths', {}) 
palsa_shapefile_path = config_paths.get('palsa_shapefile_path') # load shapefile path
save_crops_dir = config_paths.get('save_crops_dir') # load directory with all tifs
original_tif_dir = config_paths.get('original_tif_dir') # load directory with all tifs

config_img = configs.get('image_info', {}) 
dims = config_img.get('meters_per_axis') # load shapefile path

# logger.info('Configurations were loaded')

##########
## code ##
##########

# logger.info('Starting to sample relevant TIF paths...')

# extract tif file names which contain palsa
palsa_tifs = filter_imgs(original_tif_dir) # returns a list of filenames to be cropped

# logger.info(f'{len(palsa_tifs)} TIF paths have been loaded!')
# logger.info('Starting to generate training samples from TIFs..')

labels = {}

print(dims, type(dims))

# load palsa shape path
for idx, img_name in enumerate(palsa_tifs):
    img_name_code = img_name.split('.')[0]
    img_path = os.path.join(original_tif_dir, img_name)
    cropping = Crop_tif(img_name_code, img_path, palsa_shapefile_path, save_crops_dir, dims, logger)

    break

#     positive_labels = cropping.crop_rutor()
#     negative_labels = cropping.crop_negatives()
#     all_labels = positive_labels | negative_labels
#     labels = labels | all_labels
#     # logger.info(f'Generated training samples from image {idx+1}/{len(palsa_tifs)}')

# label_df = pd.DataFrame.from_dict(labels, orient='index', columns = ['palsa_percentage'])
# label_df.to_csv(os.path.join(save_crops_dir, "palsa_labels.csv"))


200 <class 'int'>
200


In [5]:
orig_rutor = cropping.filtered_rutor
all_rutor = cropping.img_rutor

In [6]:
import geopandas as gpd

# Assuming you have all_rutor and orig_rutor loaded as GeoDataFrame objects
d = {'name': [i for i in range(len(all_rutor))]}
df = pd.DataFrame(d)
all_rutor_df = gpd.GeoDataFrame(d, geometry = all_rutor, crs=all_rutor.crs)


# Perform a spatial join between all_rutor and orig_rutor
joined_df = gpd.sjoin(all_rutor_df, orig_rutor, how='inner')

# Get the unique index values of the polygons in all_rutor_df that cover at least one smaller polygon
covering_polygons_index = joined_df.index.unique()

# Select the polygons from all_rutor_df that cover at least one smaller polygon
result_df = all_rutor_df.loc[covering_polygons_index]